In [1]:
### IMPORTS
import os
import glob
import json
#import util

import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn.init as init

from collections import defaultdict
from collections import Counter

from datetime import datetime

import SimpleITK as sitk

In [2]:
saved_model_path = r'C:\Users\Leo\Documents\UHN-MedImg3D-ML-quiz\nnUNet_results\Dataset001_Pancreas\nnUNetTrainer__nnUNetPlans__2d\fold_0\checkpoint_best.pth'

# all the model architecture information is in the plans.json file:
plans_json_path = r'C:\Users\Leo\Documents\UHN-MedImg3D-ML-quiz\nnUNet_results\Dataset001_Pancreas\nnUNetTrainer__nnUNetPlans__2d\plans.json'

In [3]:
pickle = torch.load(saved_model_path, weights_only = False)

network_weights = pickle['network_weights']

In [4]:
for key, value in pickle.items():
    print(key)

network_weights
optimizer_state
grad_scaler_state
logging
_best_ema
current_epoch
init_args
trainer_name
inference_allowed_mirroring_axes


In [5]:
for key, value in network_weights.items():
    print(key)

encoder.stages.0.0.convs.0.conv.weight
encoder.stages.0.0.convs.0.conv.bias
encoder.stages.0.0.convs.0.norm.weight
encoder.stages.0.0.convs.0.norm.bias
encoder.stages.0.0.convs.0.all_modules.0.weight
encoder.stages.0.0.convs.0.all_modules.0.bias
encoder.stages.0.0.convs.0.all_modules.1.weight
encoder.stages.0.0.convs.0.all_modules.1.bias
encoder.stages.0.0.convs.1.conv.weight
encoder.stages.0.0.convs.1.conv.bias
encoder.stages.0.0.convs.1.norm.weight
encoder.stages.0.0.convs.1.norm.bias
encoder.stages.0.0.convs.1.all_modules.0.weight
encoder.stages.0.0.convs.1.all_modules.0.bias
encoder.stages.0.0.convs.1.all_modules.1.weight
encoder.stages.0.0.convs.1.all_modules.1.bias
encoder.stages.1.0.convs.0.conv.weight
encoder.stages.1.0.convs.0.conv.bias
encoder.stages.1.0.convs.0.norm.weight
encoder.stages.1.0.convs.0.norm.bias
encoder.stages.1.0.convs.0.all_modules.0.weight
encoder.stages.1.0.convs.0.all_modules.0.bias
encoder.stages.1.0.convs.0.all_modules.1.weight
encoder.stages.1.0.convs.0

In [6]:
#encoder is a module list
#stages is a module list
#stage is a module list

#convs is a module list
# conv is a module
# norm is a module
# all_modules is a module list
#    0
#    1


In [7]:
with open(plans_json_path) as f:
    plans_json_dict = json.load(f)

In [8]:
for key, value in plans_json_dict.items():
    print(key)

dataset_name
plans_name
original_median_spacing_after_transp
original_median_shape_after_transp
image_reader_writer
transpose_forward
transpose_backward
configurations
experiment_planner_used
label_manager
foreground_intensity_properties_per_channel


In [9]:
for key, value in plans_json_dict['configurations']['2d'].items():
    print(key)

data_identifier
preprocessor_name
batch_size
patch_size
median_image_size_in_voxels
spacing
normalization_schemes
use_mask_for_norm
resampling_fn_data
resampling_fn_seg
resampling_fn_data_kwargs
resampling_fn_seg_kwargs
resampling_fn_probabilities
resampling_fn_probabilities_kwargs
architecture
batch_dice


In [10]:
arch_kwargs = plans_json_dict['configurations']['2d']['architecture']['arch_kwargs']

for key, value in arch_kwargs.items():
    print(key)



n_stages
features_per_stage
conv_op
kernel_sizes
strides
n_conv_per_stage
n_conv_per_stage_decoder
conv_bias
norm_op
norm_op_kwargs
dropout_op
dropout_op_kwargs
nonlin
nonlin_kwargs


In [11]:
print(arch_kwargs)


{'n_stages': 6, 'features_per_stage': [32, 64, 128, 256, 512, 512], 'conv_op': 'torch.nn.modules.conv.Conv2d', 'kernel_sizes': [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]], 'strides': [[1, 1], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]], 'n_conv_per_stage': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'conv_bias': True, 'norm_op': 'torch.nn.modules.instancenorm.InstanceNorm2d', 'norm_op_kwargs': {'eps': 1e-05, 'affine': True}, 'dropout_op': None, 'dropout_op_kwargs': None, 'nonlin': 'torch.nn.LeakyReLU', 'nonlin_kwargs': {'inplace': True}}


In [13]:
from nnunetv2.utilities.get_network_from_plans import get_network_from_plans

# create the classification model
classify_model = get_network_from_plans(
        arch_class_name='dynamic_network_architectures.architectures.unet.ClassifyinnUNet',
        arch_kwargs=arch_kwargs,
        arch_kwargs_req_import=["conv_op", "norm_op", "dropout_op", "nonlin"],
        input_channels=1,
        output_channels=3,
        allow_init=True,
        deep_supervision=False)

classify_model = classify_model.cuda()



In [15]:
data = torch.rand(3, 1, 128, 192)

data = data.cuda()
outputs = classify_model(data) # this should be a list of torch.Tensor

print(outputs.shape)

torch.Size([3, 3])


In [23]:
print(classify_model.compute_conv_feature_map_size([4, 6]))

2560


In [19]:
for name, param in classify_model.named_parameters():
    print(name, param.shape)

encoder.stages.0.0.convs.0.conv.weight torch.Size([32, 1, 3, 3])
encoder.stages.0.0.convs.0.conv.bias torch.Size([32])
encoder.stages.0.0.convs.0.norm.weight torch.Size([32])
encoder.stages.0.0.convs.0.norm.bias torch.Size([32])
encoder.stages.0.0.convs.1.conv.weight torch.Size([32, 32, 3, 3])
encoder.stages.0.0.convs.1.conv.bias torch.Size([32])
encoder.stages.0.0.convs.1.norm.weight torch.Size([32])
encoder.stages.0.0.convs.1.norm.bias torch.Size([32])
encoder.stages.0.0.classify_convs.0.conv.weight torch.Size([32, 1, 3, 3])
encoder.stages.0.0.classify_convs.0.conv.bias torch.Size([32])
encoder.stages.0.0.classify_convs.0.norm.weight torch.Size([32])
encoder.stages.0.0.classify_convs.0.norm.bias torch.Size([32])
encoder.stages.0.0.classify_convs.1.conv.weight torch.Size([32, 32, 3, 3])
encoder.stages.0.0.classify_convs.1.conv.bias torch.Size([32])
encoder.stages.0.0.classify_convs.1.norm.weight torch.Size([32])
encoder.stages.0.0.classify_convs.1.norm.bias torch.Size([32])
encoder.st

In [26]:
classify_model_weights = classify_model.state_dict()

for key, value in classify_model_weights.items():
    print(key, value.shape)

encoder.stem.convs.0.conv.weight torch.Size([32, 1, 1, 3, 3])
encoder.stem.convs.0.conv.bias torch.Size([32])
encoder.stem.convs.0.norm.weight torch.Size([32])
encoder.stem.convs.0.norm.bias torch.Size([32])
encoder.stem.convs.0.all_modules.0.weight torch.Size([32, 1, 1, 3, 3])
encoder.stem.convs.0.all_modules.0.bias torch.Size([32])
encoder.stem.convs.0.all_modules.1.weight torch.Size([32])
encoder.stem.convs.0.all_modules.1.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.conv.weight torch.Size([32, 32, 1, 3, 3])
encoder.stages.0.blocks.0.conv1.conv.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.norm.weight torch.Size([32])
encoder.stages.0.blocks.0.conv1.norm.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.all_modules.0.weight torch.Size([32, 32, 1, 3, 3])
encoder.stages.0.blocks.0.conv1.all_modules.0.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.all_modules.1.weight torch.Size([32])
encoder.stages.0.blocks.0.conv1.all_modules.1.bias torch.Size([32])
encode

In [23]:
pickle = torch.load(saved_model_path, weights_only = False)
segmentation_model_weights = pickle['network_weights']

for key, value in segmentation_model_weights.items():
    print(key, value.shape)

encoder.stem.convs.0.conv.weight torch.Size([32, 1, 1, 3, 3])
encoder.stem.convs.0.conv.bias torch.Size([32])
encoder.stem.convs.0.norm.weight torch.Size([32])
encoder.stem.convs.0.norm.bias torch.Size([32])
encoder.stem.convs.0.all_modules.0.weight torch.Size([32, 1, 1, 3, 3])
encoder.stem.convs.0.all_modules.0.bias torch.Size([32])
encoder.stem.convs.0.all_modules.1.weight torch.Size([32])
encoder.stem.convs.0.all_modules.1.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.conv.weight torch.Size([32, 32, 1, 3, 3])
encoder.stages.0.blocks.0.conv1.conv.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.norm.weight torch.Size([32])
encoder.stages.0.blocks.0.conv1.norm.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.all_modules.0.weight torch.Size([32, 32, 1, 3, 3])
encoder.stages.0.blocks.0.conv1.all_modules.0.bias torch.Size([32])
encoder.stages.0.blocks.0.conv1.all_modules.1.weight torch.Size([32])
encoder.stages.0.blocks.0.conv1.all_modules.1.bias torch.Size([32])
encode

In [20]:
# load the segmentation model weights
pickle = torch.load(saved_model_path, weights_only = False)
segmentation_model_weights = pickle['network_weights']

# get the weights of the classification model (with random initializations)
classify_model_weights = classify_model.state_dict()

# identify which layers are both in the segmentation encoder and classification encoder
common_keys = classify_model_weights.keys() & segmentation_model_weights.keys()
common_keys = list(common_keys)
common_keys.sort()

# replace the weights of the classification model with pre-trained segmentation model weights
for key in common_keys:
    classify_model_weights[key] = segmentation_model_weights[key]
    
    classify_key = key.replace('convs', 'classify_convs')
    classify_model_weights[classify_key] = segmentation_model_weights[key]

# update the model
classify_model.load_state_dict(classify_model_weights)



RuntimeError: Error(s) in loading state_dict for ResidualClassifyinnUNet:
	Unexpected key(s) in state_dict: "encoder.stem.classify_convs.0.all_modules.0.bias", "encoder.stem.classify_convs.0.all_modules.0.weight", "encoder.stem.classify_convs.0.all_modules.1.bias", "encoder.stem.classify_convs.0.all_modules.1.weight", "encoder.stem.classify_convs.0.conv.bias", "encoder.stem.classify_convs.0.conv.weight", "encoder.stem.classify_convs.0.norm.bias", "encoder.stem.classify_convs.0.norm.weight". 

In [ ]:
for name, param in classify_model.named_parameters():
    if 'convs' in name and 'classify' not in name:
        param.requires_grad = False
    #if name contains convs
    # then freeze param

    # if name contains convs but not contains classify
    # freeze param
    

In [42]:
for name, param in classify_model.named_parameters():
    print(name, param.requires_grad)
        

encoder.stages.0.0.convs.0.conv.weight False
encoder.stages.0.0.convs.0.conv.bias False
encoder.stages.0.0.convs.0.norm.weight False
encoder.stages.0.0.convs.0.norm.bias False
encoder.stages.0.0.convs.1.conv.weight False
encoder.stages.0.0.convs.1.conv.bias False
encoder.stages.0.0.convs.1.norm.weight False
encoder.stages.0.0.convs.1.norm.bias False
encoder.stages.0.0.classify_convs.0.conv.weight True
encoder.stages.0.0.classify_convs.0.conv.bias True
encoder.stages.0.0.classify_convs.0.norm.weight True
encoder.stages.0.0.classify_convs.0.norm.bias True
encoder.stages.0.0.classify_convs.1.conv.weight True
encoder.stages.0.0.classify_convs.1.conv.bias True
encoder.stages.0.0.classify_convs.1.norm.weight True
encoder.stages.0.0.classify_convs.1.norm.bias True
encoder.stages.0.0.attention_gate.conv.weight True
encoder.stages.0.0.attention_gate.conv.bias True
encoder.stages.1.0.convs.0.conv.weight False
encoder.stages.1.0.convs.0.conv.bias False
encoder.stages.1.0.convs.0.norm.weight False

In [23]:
for key in common_keys:
    print(key)

encoder.stages.0.0.convs.0.all_modules.0.bias
encoder.stages.0.0.convs.0.all_modules.0.weight
encoder.stages.0.0.convs.0.all_modules.1.bias
encoder.stages.0.0.convs.0.all_modules.1.weight
encoder.stages.0.0.convs.0.conv.bias
encoder.stages.0.0.convs.0.conv.weight
encoder.stages.0.0.convs.0.norm.bias
encoder.stages.0.0.convs.0.norm.weight
encoder.stages.0.0.convs.1.all_modules.0.bias
encoder.stages.0.0.convs.1.all_modules.0.weight
encoder.stages.0.0.convs.1.all_modules.1.bias
encoder.stages.0.0.convs.1.all_modules.1.weight
encoder.stages.0.0.convs.1.conv.bias
encoder.stages.0.0.convs.1.conv.weight
encoder.stages.0.0.convs.1.norm.bias
encoder.stages.0.0.convs.1.norm.weight
encoder.stages.1.0.convs.0.all_modules.0.bias
encoder.stages.1.0.convs.0.all_modules.0.weight
encoder.stages.1.0.convs.0.all_modules.1.bias
encoder.stages.1.0.convs.0.all_modules.1.weight
encoder.stages.1.0.convs.0.conv.bias
encoder.stages.1.0.convs.0.conv.weight
encoder.stages.1.0.convs.0.norm.bias
encoder.stages.1.0.

In [16]:
# test to see if they are the same
test_classify_model_weights = classify_model.state_dict()

print(test_classify_model_weights['encoder.stages.0.0.convs.0.conv.weight'])


tensor([[[[[-7.2278e-02, -3.3149e-02, -4.2815e-02],
           [-7.2781e-02, -9.2624e-02,  2.9016e-01],
           [ 4.4994e-01, -4.1597e-01,  4.4787e-01]]]],



        [[[[-5.2829e-01,  9.0586e-02,  4.3066e-01],
           [-6.6217e-01,  7.9920e-02,  6.7638e-01],
           [-3.7028e-01, -1.8988e-01,  5.3336e-01]]]],



        [[[[ 3.8833e-01,  4.5634e-01,  2.3674e-01],
           [-1.0797e-01,  4.0362e-01,  1.4066e-01],
           [ 3.3304e-03, -7.0152e-01, -9.2850e-01]]]],



        [[[[-6.4078e-01, -5.8295e-01,  4.7005e-01],
           [-3.2339e-01, -5.3966e-01,  3.3987e-01],
           [-5.8612e-02,  5.8490e-01,  7.2286e-01]]]],



        [[[[ 2.1391e-01,  8.6111e-02,  2.2119e-01],
           [ 3.3159e-01,  3.7024e-01, -1.9706e-01],
           [-1.4893e-02,  1.6651e-01, -4.8422e-01]]]],



        [[[[-1.1574e-01,  2.4167e-01, -3.8216e-01],
           [-3.8319e-01,  3.9457e-01,  1.7448e-03],
           [ 5.2181e-02,  1.0879e-02, -2.8775e-01]]]],



        [[[[ 2.4068e-01, -3.

In [17]:
print(segmentation_model_weights['encoder.stages.0.0.convs.0.conv.weight'])


tensor([[[[[-7.2278e-02, -3.3149e-02, -4.2815e-02],
           [-7.2781e-02, -9.2624e-02,  2.9016e-01],
           [ 4.4994e-01, -4.1597e-01,  4.4787e-01]]]],



        [[[[-5.2829e-01,  9.0586e-02,  4.3066e-01],
           [-6.6217e-01,  7.9920e-02,  6.7638e-01],
           [-3.7028e-01, -1.8988e-01,  5.3336e-01]]]],



        [[[[ 3.8833e-01,  4.5634e-01,  2.3674e-01],
           [-1.0797e-01,  4.0362e-01,  1.4066e-01],
           [ 3.3304e-03, -7.0152e-01, -9.2850e-01]]]],



        [[[[-6.4078e-01, -5.8295e-01,  4.7005e-01],
           [-3.2339e-01, -5.3966e-01,  3.3987e-01],
           [-5.8612e-02,  5.8490e-01,  7.2286e-01]]]],



        [[[[ 2.1391e-01,  8.6111e-02,  2.2119e-01],
           [ 3.3159e-01,  3.7024e-01, -1.9706e-01],
           [-1.4893e-02,  1.6651e-01, -4.8422e-01]]]],



        [[[[-1.1574e-01,  2.4167e-01, -3.8216e-01],
           [-3.8319e-01,  3.9457e-01,  1.7448e-03],
           [ 5.2181e-02,  1.0879e-02, -2.8775e-01]]]],



        [[[[ 2.4068e-01, -3.

In [18]:
print(test_classify_model_weights['encoder.stages.0.0.convs.0.conv.weight'])

tensor([[[[[-7.2278e-02, -3.3149e-02, -4.2815e-02],
           [-7.2781e-02, -9.2624e-02,  2.9016e-01],
           [ 4.4994e-01, -4.1597e-01,  4.4787e-01]]]],



        [[[[-5.2829e-01,  9.0586e-02,  4.3066e-01],
           [-6.6217e-01,  7.9920e-02,  6.7638e-01],
           [-3.7028e-01, -1.8988e-01,  5.3336e-01]]]],



        [[[[ 3.8833e-01,  4.5634e-01,  2.3674e-01],
           [-1.0797e-01,  4.0362e-01,  1.4066e-01],
           [ 3.3304e-03, -7.0152e-01, -9.2850e-01]]]],



        [[[[-6.4078e-01, -5.8295e-01,  4.7005e-01],
           [-3.2339e-01, -5.3966e-01,  3.3987e-01],
           [-5.8612e-02,  5.8490e-01,  7.2286e-01]]]],



        [[[[ 2.1391e-01,  8.6111e-02,  2.2119e-01],
           [ 3.3159e-01,  3.7024e-01, -1.9706e-01],
           [-1.4893e-02,  1.6651e-01, -4.8422e-01]]]],



        [[[[-1.1574e-01,  2.4167e-01, -3.8216e-01],
           [-3.8319e-01,  3.9457e-01,  1.7448e-03],
           [ 5.2181e-02,  1.0879e-02, -2.8775e-01]]]],



        [[[[ 2.4068e-01, -3.

In [19]:
print(classify_model_weights['encoder.stages.0.0.classify_convs.0.conv.weight'])

tensor([[[[[-7.2278e-02, -3.3149e-02, -4.2815e-02],
           [-7.2781e-02, -9.2624e-02,  2.9016e-01],
           [ 4.4994e-01, -4.1597e-01,  4.4787e-01]]]],



        [[[[-5.2829e-01,  9.0586e-02,  4.3066e-01],
           [-6.6217e-01,  7.9920e-02,  6.7638e-01],
           [-3.7028e-01, -1.8988e-01,  5.3336e-01]]]],



        [[[[ 3.8833e-01,  4.5634e-01,  2.3674e-01],
           [-1.0797e-01,  4.0362e-01,  1.4066e-01],
           [ 3.3304e-03, -7.0152e-01, -9.2850e-01]]]],



        [[[[-6.4078e-01, -5.8295e-01,  4.7005e-01],
           [-3.2339e-01, -5.3966e-01,  3.3987e-01],
           [-5.8612e-02,  5.8490e-01,  7.2286e-01]]]],



        [[[[ 2.1391e-01,  8.6111e-02,  2.2119e-01],
           [ 3.3159e-01,  3.7024e-01, -1.9706e-01],
           [-1.4893e-02,  1.6651e-01, -4.8422e-01]]]],



        [[[[-1.1574e-01,  2.4167e-01, -3.8216e-01],
           [-3.8319e-01,  3.9457e-01,  1.7448e-03],
           [ 5.2181e-02,  1.0879e-02, -2.8775e-01]]]],



        [[[[ 2.4068e-01, -3.

In [20]:
data = torch.rand(3, 1, 64, 128, 192)

data = data.cuda()
outputs = classify_model(data) # this should be a list of torch.Tensor

In [23]:
print(outputs)


tensor([[-0.0111,  0.0458, -0.0181],
        [-0.0086,  0.0299, -0.0497],
        [-0.0100,  0.0515, -0.0363]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [27]:
lesions = torch.tensor([[0., 1., 0.], 
 [1., 0., 0.], 
 [0., 0., 1.]])
lesions = lesions.cuda()

In [28]:
loss_fx = nn.CrossEntropyLoss()

In [38]:
predict = torch.tensor([[.05, .9, .05], 
 [.9, .05, .05], 
 [0., 0., 1.]])
predict = predict.cuda()

In [39]:
print(loss_fx(predict, lesions))

tensor(0.5957, device='cuda:0')


In [6]:

# ENCODERS and DECODER-ENCODER and DECODERS:

# there are 3 types: encoder, decoder.encoder, and decoder
#encoder.stages.0.0.convs.0.conv.weight
#decoder.encoder.stages.0.0.convs.0.conv.weight
#decoder.stages.0.convs.0.conv.weight


# for encoder and decoder.encoder, there are 6 stages (0 to 5):
#encoder.stages.0.0.convs.0.conv.weight
#...
#encoder.stages.5.0.convs.0.conv.weight
#          and
#decoder.encoder.stages.0.0.convs.0.conv.weight
#...
#decoder.encoder.stages.5.0.convs.0.conv.weight


## each stage has two convs: 
#encoder.stages.0.0.convs.0.conv.weight
#encoder.stages.0.0.convs.1.conv.weight

## and each stage has these modules:
#encoder.stages.0.0.convs.0.conv.weight
#encoder.stages.0.0.convs.0.conv.bias
#encoder.stages.0.0.convs.0.norm.weight
#encoder.stages.0.0.convs.0.norm.bias
#encoder.stages.0.0.convs.0.all_modules.0.weight
#encoder.stages.0.0.convs.0.all_modules.0.bias
#encoder.stages.0.0.convs.0.all_modules.1.weight
#encoder.stages.0.0.convs.0.all_modules.1.bias



# for decoder, there are 5 stages (0 to 4)
#   for each stage, there are two convs (0 and 1)
     # decoder.stages.0.convs.0.conv.weight
     # decoder.stages.1.convs.0.conv.weight
     # these have the modules
            #decoder.stages.0.convs.0.conv.weight
            #decoder.stages.0.convs.0.conv.bias
            #decoder.stages.0.convs.0.norm.weight
            #decoder.stages.0.convs.0.norm.bias
            #decoder.stages.0.convs.0.all_modules.0.weight
            #decoder.stages.0.convs.0.all_modules.0.bias
            #decoder.stages.0.convs.0.all_modules.1.weight
            #decoder.stages.0.convs.0.all_modules.1.bias

   # there are the transpconvs and seg_layers, one per decoder stage
        #decoder.transpconvs.0.weight  and bias
        #...
        #decoder.transpconvs.4.weight
        #decoder.transpconvs.4.bias
       
        
        #decoder.seg_layers.0.weight
        #decoder.seg_layers.0.bias
        #...
        #decoder.seg_layers.4.weight
        #decoder.seg_layers.4.bias




In [ ]:
n_stages = arch_kwargs['n_stages']
features_per_stage = arch_kwargs['features_per_stage']
kernel_sizes = arch_kwargs['kernel_sizes']
strides = arch_kwargs['strides']

n_conv_per_stage = arch_kwargs['n_conv_per_stage']
n_conv_per_stage_decoder = arch_kwargs['n_conv_per_stage_decoder']

# can I use these?
#conv_op = arch_kwargs['conv_op']
#norm_op = arch_kwargs['norm_op']


# encoders

# for encoder and decoder.encoder, there are 6 stages (0 to 5):
#encoder.stages.0.0.convs.0.conv.weight
#...
#encoder.stages.5.0.convs.0.conv.weight
#          and
#decoder.encoder.stages.0.0.convs.0.conv.weight
#...
#decoder.encoder.stages.5.0.convs.0.conv.weight


## each stage has two convs: 
#encoder.stages.0.0.convs.0.conv.weight
#encoder.stages.0.0.convs.1.conv.weight

## and each stage has these modules:
#encoder.stages.0.0.convs.0.conv.weight
#encoder.stages.0.0.convs.0.conv.bias
#encoder.stages.0.0.convs.0.norm.weight
#encoder.stages.0.0.convs.0.norm.bias
#encoder.stages.0.0.convs.0.all_modules.0.weight
#encoder.stages.0.0.convs.0.all_modules.0.bias
#encoder.stages.0.0.convs.0.all_modules.1.weight
#encoder.stages.0.0.convs.0.all_modules.1.bias
